<a href="https://colab.research.google.com/github/mcupk/FaceDetection/blob/main/AlexNet_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect My Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/My Drive/Colab Notebooks/Dataset/tarball-lite/AFAD-Lite/

/content/drive/My Drive/Colab Notebooks/Dataset/tarball-lite/AFAD-Lite


# Creating Custom Dataset


In [ ]:
from torch.utils.data import Dataset
from  PIL import Image
import pandas as pd
import os
import torch
from skimage import io

class FaceData(Dataset):
  def __init__(self, root_dir, csv_file, transform=None):
    self.annotations = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform

  def __len__(self):
    return len(self.annotations)

  def __getitem__(self, index):
    # img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 1])
    # img = io.imread(img_path)
    img_id = self.annotations.iloc[index, 1]
    img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")
    y_label = self.annotations.iloc[index, 2:]
    y_label = y_label.astype("float", copy=False)
    y_label = torch.tensor(y_label)
    if self.transform:
      img = self.transform(img)
    
    return (img, y_label)

# Creating Model

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class AlexNet(nn.Module):
  def __init__(self, train_AlexNet=False, num_classes=21):
    super(AlexNet, self).__init__()
    self.train_CNN = train_CNN
    self.alexnet = torch.hub.load('pytorch/vision:v0.9.0', 'alexnet', pretrained=True)
    self.alexnet.classifier[6] = nn.Linear(4096, num_classes)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(0.5)
    self.linear = nn.Sequential()
    # self.linear2 = nn.Linear(32*73*73, 21)
    self.alexnet.eval()

  def forward(self, x):
    features = self.alexnet(x)
  
    return features
    return self.linear(self.dropout(self.relu(features))).squeeze(1)

    # return out

# Importing Libraries for training Loop

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
import numpy as np

# import tqdm

device = ("cuda" if torch.cuda.is_available() else "cpu")

# Transformations

In [ ]:
transform = transforms.Compose([
        transforms.Resize((356, 356)),
        transforms.RandomCrop((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
])

# Hyperparameters

In [ ]:
num_epochs = 15
learning_rate = 0.001
train_CNN = False
batch_size = 32
shuffle = True
pin_memory = True
num_workers = 2 

# Setting the dataset and dataloader

In [ ]:
dataset = FaceData("train", "New_train.csv", transform=transform)
lenghts = [int(round(len(dataset)*0.7)),int(round(len(dataset)*0.2)),
           int(round(len(dataset)*0.1))]
train_set, validation_set, test_set = torch.utils.data.random_split(dataset, lenghts)
train_loader = DataLoader(dataset=train_set, shuffle=shuffle,
                          batch_size=batch_size, num_workers=num_workers,
                          pin_memory=pin_memory)
validation_loader = DataLoader(dataset=validation_set, shuffle=shuffle,
                               batch_size=batch_size,num_workers=num_workers,
                               pin_memory=pin_memory)
test_loader = DataLoader(dataset=test_set, shuffle=shuffle,
                          batch_size=batch_size, num_workers=num_workers,
                          pin_memory=pin_memory)

In [ ]:
model = AlexNet().to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5,gamma=0.1)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0


# Accuracy Check

In [ ]:
def check_accuracy(loader, model):
  if loader == train_loader:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on validation data")

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
        x = x.to(device=device)
        y = y.to(device=decice)

        scores = model(x)
        predictions = torch.tensor([1.0 if i >= 0.5 else 0.0 for i in scores]).to(device)
        num_correct += (predictions == y).sum()
        num_samples += predictions.size(0)
  return f"{float(num_correct)/float(num_samples)*100:.2f}"

  print(
      f"Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
  )

  model.train()

# Train


In [ ]:
def train():
  model.train()
  for epoch in range(num_epochs):
    
    training_loss = 0.0
    val_loss = 0.0
    val_acc = 0
    validation = 0.0
    total = 0
    correct_preds = 0
    total_train = 0
    correct_train = 0
    train_acc = 0
    
    loop = tqdm(train_loader, total=len(train_loader), leave=True)

    for imgs, labels in loop:
        imgs = imgs.to(device)
        labels = labels.to(device)
        outputs = model(imgs)
        
        
        total_train += labels.size(0)
        _, prediction_train = torch.max(outputs.data, 1)
        _, y_labels = torch.max(labels.data, 1)
        # print(prediction_train.type())
        # print(y_labels.type())
        correct_train += (prediction_train == y_labels).sum().item()
        # print('correct_train',correct_train)

        loss = criterion(outputs.float(),labels.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        training_loss += loss.item()
       
        
    valid_data_loader = tqdm(validation_loader, total=len(validation_loader), leave=True)   

    with torch.no_grad():
      model.eval()
      for imgs, labels in valid_data_loader:
          imgs = imgs.to(device)
          labels = labels.to(device)
          outputs = model(imgs)

          val_loss = criterion(outputs, labels) 
          validation += val_loss.item()

          total += labels.size(0)
          _, prediction = torch.max(outputs.data, 1)
          # print('predicted: ', prediction)
          _, y_labels = torch.max(labels, 1)
          correct_preds += (prediction==y_labels).sum().item()
          # print('correct_preds: ', correct_preds)
          # print('prediction == labels: ', (prediction == labels).sum().item())
          # print('labels: ', labels)
           
      train_acc = 100 *(correct_train / total_train)    
      val_acc = 100 * (correct_preds / total)
      

      training_loss /= len(loop)
      validation /= len(valid_data_loader)
      print(f'Epoch [{epoch}/{num_epochs}]  Training loss: {training_loss}.. Validation Loss: {validation}')
      print('Training Accuracy is : {:.2f}%'.format(train_acc))
      print('Validation Accuracy is : {:.2f}%'.format(val_acc))

if __name__ == "__main__":
  train()

  0%|          | 0/1259 [00:00<?, ?it/s]

Epoch [0/15]  Training loss: 0.04563292582845574.. Validation Loss: 0.045240732085047856
Training Accuracy is : 8.01%
Validation Accuracy is : 7.68%


  0%|          | 0/1259 [00:00<?, ?it/s]

Epoch [1/15]  Training loss: 0.04525695925517817.. Validation Loss: 0.04524900178607501
Training Accuracy is : 7.33%
Validation Accuracy is : 7.51%


  0%|          | 0/1259 [00:00<?, ?it/s]

...

Epoch [12/15]  Training loss: 0.04525695883205147.. Validation Loss: 0.04523665799617101
Training Accuracy is : 7.57%
Validation Accuracy is : 7.59%


  0%|          | 0/1259 [00:00<?, ?it/s]

Epoch [13/15]  Training loss: 0.045256225515017914.. Validation Loss: 0.045241968012066466
Training Accuracy is : 7.39%
Validation Accuracy is : 7.47%


100%|██████████| 360/360 [01:07<00:00,  5.36it/s]

Epoch [14/15]  Training loss: 0.04525775512394875.. Validation Loss: 0.04525474617168488
Training Accuracy is : 7.36%
Validation Accuracy is : 7.36%
